In [ ]:
import random

import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import img_to_array, load_img # type: ignore

In [ ]:
# Set Variables
DATA_PATH = Path("../data/fungi-clef-2025")
MD_PATH = DATA_PATH / "metadata/FungiTastic-FewShot"
IMAGE_PATH = DATA_PATH / "images/FungiTastic-FewShot"
LABEL = "class"

### Data Processing

In [ ]:
# Load the metadata for each split
md_train_val = pd.read_csv(MD_PATH / "FungiTastic-FewShot-Train.csv")
# Use the validation set as the test set because it has labels
md_test = pd.read_csv(MD_PATH / "FungiTastic-FewShot-Val.csv")

In [ ]:
# Drop any rows that are missing labels
md_train_val = md_train_val.dropna(subset=LABEL)
# Drop any classes that have only 1 label so we can stratify
md_train_val = md_train_val.groupby(LABEL).filter(lambda group: len(group) > 1)

In [ ]:
# Split the train data into training and validation
md_train, md_val = train_test_split(md_train_val, test_size=0.20, stratify=md_train_val[LABEL])

In [ ]:
# Label each split
md_train["split"] = "train"
md_val["split"] = "val"
md_test["split"] = "test"

In [ ]:
# Join all of the data together
md_df = pd.concat([md_train, md_val, md_test])

# Add the full image location for each image
# Options for image size include 300p, 500p, 720p, fullsize
md_df["image_path"] = md_df.apply(
    lambda row: IMAGE_PATH / f"{'val' if row['split'] == 'test' else 'train'}/300p/{row['filename']}", axis=1
)

In [ ]:
# Map the class to an ID
le = preprocessing.LabelEncoder()
le.fit(md_df["class"])
md_df["class_label"] = md_df["class"]
md_df["class_idx"] = le.transform(md_df["class"])

### Image Loading

In [ ]:
images = []
labels = []

TARGET_SIZE = 224

label_counts = {}

for _, row in md_df.iterrows():
    #
    if row["class_idx"] not in label_counts:
        label_counts[row["class_idx"]] = 0

    if label_counts[row["class_idx"]] >= 1000:
        pass

    label_counts[row["class_idx"]] += 1

    # Load and save the image as an array
    img = load_img(row["image_path"])

    # Get original dimensions
    original_height = tf.cast(tf.shape(img)[0], tf.float32)
    original_width = tf.cast(tf.shape(img)[1], tf.float32)

    # Calculate scaling factor to maintain aspect ratio
    height_scale = TARGET_SIZE / original_height
    width_scale = TARGET_SIZE / original_width
    scale = tf.minimum(height_scale, width_scale)

    # Calculate new dimensions
    new_height = tf.cast(tf.math.round(original_height * scale), tf.int32)
    new_width = tf.cast(tf.math.round(original_width * scale), tf.int32)

    # Resize the image while maintaining aspect ratio
    resized_img = tf.image.resize(img, [new_height, new_width], method='bilinear')

    # Use resize_with_pad to add padding to make the image square
    padded_img = tf.image.resize_with_pad(
        resized_img, 
        TARGET_SIZE, 
        TARGET_SIZE, 
        method='bilinear'
    )

    img_arr = img_to_array(padded_img)
    images.append(img_arr)

    # Append the class to the list of labels
    labels.append(row["class_idx"])

    {label: train_labels_class.count(label) for label in set(train_labels_class)}

In [ ]:
def apply_random_augmentation(X):
    random_number = random.randint(1, 4)

    if random_number == 1:
        return tf.image.flip_left_right(X)
    if random_number == 2:
        return tf.image.flip_up_down(X)
    if random_number == 3:
        return tf.image.adjust_brightness(X, delta=0.3)
    if random_number == 4:
        return tf.image.adjust_contrast(X, contrast_factor=3)


In [ ]:
augmented_images = []
augmented_labels = []

for class_label, label_count in label_counts.items():
    # The number of images that need to be augmented and appended to reach 1000
    images_to_augment = 1000 - label_count
    
    # Pool of potential images to augment
    image_pool_idx = [i for i, label in enumerate(labels) if label == class_label]

    for i in range(images_to_augment):

        # Select a random image to augment
        image_idx = random.choice(image_pool_idx)
        image_to_aug = images[image_idx]

        # Apply a random augmentation
        augmented = apply_random_augmentation(image_to_aug)

        # Save new image and label
        augmented_images.append(augmented)
        augmented_labels.append(class_label)

In [ ]:
# Add to the existing
images = images + augmented_images
labels = labels + augmented_labels

In [ ]:
# Stack and convert into a numpy array
images = np.stack(images)

In [ ]:
# Rescale all of the images so they're pixel value is between [0, 1]
images = images / 255.0

In [ ]:
# Cast label list to np.array for easier manipulation
labels = np.array(labels)

In [ ]:
# Re-split the images and their labels
train_idx = md_df["split"] == "train"
val_idx = md_df["split"] == "val"
test_idx = md_df["split"] == "test"

train_images = images[train_idx]
train_labels = labels[train_idx]

val_images = images[val_idx]
val_labels = labels[val_idx]

test_images = images[test_idx]
test_labels = labels[test_idx]

In [ ]:
print(f"Shape train images: {train_images.shape}")
print(f"Shape val images: {val_images.shape}")
print(f"Shape test images: {test_images.shape}")

In [ ]:
SAVE_AS = Path("./balanced_classes")
SAVE_AS.mkdir(exist_ok=True)

# Save the image data
np.save(SAVE_AS / "train_images.npy", train_images)
np.save(SAVE_AS / "val_images.npy", val_images)
np.save(SAVE_AS / "test_images.npy", test_images)

# Save the label data
np.save(SAVE_AS / "train_labels.npy", train_labels)
np.save(SAVE_AS / "val_labels.npy", val_labels)
np.save(SAVE_AS / "test_labels.npy", test_labels)

# Save the  metadata
md_df.to_csv(SAVE_AS / "metadata.csv")
